In [1]:
import os
import json
from mappings import classToAnsible, ansibleToClass # need to get the mapping to ansible from json
import yaml
import pprint
from isConfigurableMap import isConfigurableMap # need to get the 
from collections import Counter
from requiredParamsAliasesMap import requiredParamsAliases
from collections import deque
from defaultClassAttrValues import defaults

In [2]:
# load the json file

with open('tn-nils-4.json', 'r') as file:
    data = json.load(file)

In [3]:
# function to map to ansible according to mappings.py

def map_json_to_ansible(json_data, key, map):
    try:
        new_key = map[key]
        json_data[new_key] = json_data.pop(key) # replace with ansible term
    except(KeyError): # no match found, skip
        pass

In [4]:
def remove_isNotConfigurable(key, delete_key_list):
    try:
        if isConfigurableMap[key] == False:
            delete_key_list.append(key)
        else:
            pass
    except(KeyError):
        pass

In [5]:
# this is mostly chatGPT, will need to rewrite it and figure out how to do this without it

names = []

def prune_dict(data):
    def contains_name_key(d):
        # Check if 'name' key is in the current dictionary
        if 'name' in d:
            return True
        # Check nested dictionaries
        for key, value in d.items():
            if isinstance(value, dict) and contains_name_key(value):
                return True
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict) and contains_name_key(item):
                        return True
        return False

    def prune(d):
        keys_to_delete = []
        for key, value in d.items():
            if isinstance(value, dict):
                if not contains_name_key(value):
                    keys_to_delete.append(key)
                else:
                    prune(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        prune(item)

        for key in keys_to_delete:
            del d[key]

    if isinstance(data, dict):
        prune(data)
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                prune(item)

# maybe only modules appear in the playbooks, not classes (?)

In [6]:
# we need a (helper) method that can parse the exception list and check if all params are included correctly

def enforceIfRequired(data, parent_key = None, stack = deque(maxlen = 2), required_if_map = requiredParamsAliases):

    if isinstance(data, dict):
        for key, value in data.items():
            # this is where everything happens (the recursion just leads us here every time)
            print(key, parent_key)
            
            # first append the parent key of children to the stack
            # doubt we even need a stack here, maybe just list[0] suffices
            if key == "children":
                if len(stack) > 1:
                    stack.popleft()
                stack.append(parent_key)
                
            # if key == "attributes":
                

            if isinstance(value, dict):

                enforceIfRequired(value, key) # pass key as parent_key

            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, (dict, list)):

                        enforceIfRequired(item, key) # pass key as parent_key

    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):

                enforceIfRequired(item, parent_key)

In [7]:
def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', ""]

    # process data recursively
    def process_data(data):

        if isinstance(data, dict):
            keys_to_delete = [] # need this otherwise we get runtime error (dictionary changed size etc)

            for key in data.keys():

                # if value is in default arguments, we can delete it
                if data[key] in default_args:
                    keys_to_delete.append(key)

                else:
                    # nested dictionaries
                    if isinstance(data[key], (dict, list)):
                        try:
                            process_data(data[key])
                        except (KeyError, TypeError) as e:
                            print(f"Error processing key {key}: {e}")

                # uncomment this if we want to delete all non-configurable modules
                remove_isNotConfigurable(key, keys_to_delete)

            # keys to delete
            for key in keys_to_delete:
                del data[key]

            # use the mapping function
            for key in list(data.keys()):
                map_json_to_ansible(data, key, classToAnsible)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process_data(item)

    # Process the initial data
    process_data(data)
    
    # prune_dict(data)

    # Save the resulting dictionary to a YAML file
    with open(save_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    print(f"YAML file has been saved to {save_path}")
    
    return data

In [8]:
reconstructed = reconstruct_yml(data)

YAML file has been saved to ansible_reconstructed.yml


In [9]:
reconstructed

{'aci_tenant': {'attributes': {'annotation': 'orchestrator:ansible',
   'dn': 'uni/tn-nils',
   'name': 'nils',
   'userdom': ':all:'},
  'children': [{'vnsSvcCont': {'attributes': {'userdom': 'all'}}},
   {'fvRsTenantMonPol': {'attributes': {'userdom': 'all'}}},
   {'fvEpTags': {'attributes': {'userdom': 'all'}}},
   {'aci_vrf': {'attributes': {'annotation': 'orchestrator:ansible',
      'bdEnforcedEnable': 'no',
      'descr': 'marcel',
      'ipDataPlaneLearning': 'enabled',
      'knwMcastAct': 'permit',
      'name': 'bob',
      'pcEnfDir': 'ingress',
      'pcEnfPref': 'enforced',
      'userdom': ':all:',
      'vrfIndex': '0'},
     'children': [{'fvRsVrfValidationPol': {'attributes': {'userdom': 'all'}}},
      {'vzAny': {'attributes': {'matchT': 'AtleastOne',
         'prefGrMemb': 'disabled',
         'userdom': 'all'}}},
      {'fvRsOspfCtxPol': {'attributes': {'userdom': 'all'}}},
      {'fvRsCtxToEpRet': {'attributes': {'userdom': 'all'}}},
      {'fvRsCtxToExtRouteTagPo

In [10]:
def map_isConfigurable_to_ansible(data):
    for key, value in data.items():
        map_json_to_ansible(data, key)

In [11]:
with open('aci-meta-5.3-2c.json', 'r') as file:
    meta = json.load(file)

In [12]:
import os
import re
import pprint

PATH_TO_MODULE_DIR = "/Users/lpaggen/Documents/research_project/ansible-aci/modules"

def get_file_paths(dir):
    py_files = []
    for file in os.listdir(dir):
        file_path = os.path.join(dir, file)
        if file.endswith(".py"):
            py_files.append(file_path)
    return py_files

# code is bad at the moment, will address if needed, for the time being it runs well enough

def extract_parameter_aliases(dir):
    count = 0

    py_files = get_file_paths(dir)
    out = {}

    # regex 
    # finds the block of code in source code after "argument_spec.update()"
    pattern = re.compile(r'argument_spec\.update\(\s*(.*?)^\s*\)', re.DOTALL | re.MULTILINE)
    pattern2 = re.compile(r"([a-zA-Z_]+)\s*=\s*dict", re.MULTILINE)
    pattern3 = re.compile(r"([a-zA-Z_]+)\s*=\s*dict\s*\((.*?)\)", re.DOTALL)
    
    # finds the aliases of the params according to what is found in match 3
    pattern4 = re.compile(r'(?<=aliases=\[)([^\]]*)', re.DOTALL | re.MULTILINE)
    
    # finds the block of code in source after "class_config=dict()"
    pattern5 = re.compile(r'class_config\s*=\s*dict\s*\(\s*([^)]+)', re.DOTALL)
    
    # finds the kw in the pattern5
    pattern6 = re.compile(r"([^=,\s]+)=([^'\n,]*)")

    for file_path in py_files:
        with open(file_path, 'r') as file:
            file_content = file.read()

        matches = pattern.findall(file_content)
        match5 = pattern5.findall(file_content)

        entry_dict = {}  # nested dict

        for entries_str in matches:
            match2 = pattern2.findall(entries_str) # list of the parameters
            match3 = pattern3.findall(entries_str)

            if match3:
                for idx, i in enumerate(match3): # get idx to find correspondance in match2
                    match4 = pattern4.findall(str(i)) # need string otherwise error sometimes
                    if match4:
                        for j in match4:
                            entry_dict[match2[idx]] = [i.replace("\"", "").replace(" ", "") for i in j.split(",")] # clean string
                    else:
                        entry_dict[match2[idx]] = None

            # now this looks for the second set of aliases (source code seems to be weird)
            # it should overwrite the "None" values if any
            # if match5:
            #     for idx, i in enumerate(match5):
            #         temp = i.split(",")
            #         # print(file_path.split("/")[-1][:-3])
            #         for j in temp:
            #             match6 = pattern6.search(str(j))
            #             if match6:
            #                 # print(match6.group(2), match6.group(1))
            #                 try:
            #                     if entry_dict[match6.group(2)] == None:
            #                         entry_dict[match6.group(2)] = [match6.group(1)] # ",".join([i for i in match6.group(1)])
            #                     elif match6.group(2) != None and type(match6.group(2)) != str: # error somewhere
            #                         entry_dict[match6.group(2)].append(match6.group(1))
            #                 except(KeyError):
            #                     entry_dict[match6.group(2)] = [match6.group(1)]

                print("\n")

        out[file_path.split("/")[-1][:-3]] = entry_dict

    return out

In [13]:
# this parser just gets the rnFormat of each class

def getClassRnFormat(data):
    out = {}

    def process_data(data, parent_key = None):
        if isinstance(data, dict):
            for key, value in data.items():

                if key == "rnFormat":
                    try:
                        entry_key = classToAnsible[parent_key]
                    except(KeyError):
                        entry_key = parent_key

                    out[entry_key] = value if len(value) > 0 else None

                if isinstance(value, (dict, list)):
                    process_data(value, key)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process_data(item, parent_key)

    process_data(data)

    return out

In [14]:
a = getClassRnFormat(meta)

In [15]:
# save to file
with open("classRnFormat.py", 'w') as file:
    file.write("rnFormat = ")
    pprint.pprint(a, stream=file)

print(f"Dictionary has been saved to classRnFormat.py")

Dictionary has been saved to classRnFormat.py


# need way to decide which mapping, according to the parent in the json

In [16]:
exception_list=['spanDestGrp',
                'spanSrcGrp',
                'spanSrc',
                'mgmtMaintP',
                'spanRsSrcToPathEp',
                'fvSubnet',
                'fvRsPathAtt',
                'vzBrCP',
                'dhcpRelayP',
                'infraRsVlanNs',
                'fvRsSecInherited',
                'l1PhysIf',
                'l2extInstP',
                'mgmtOoB']

In [17]:
Counter(ansibleToClass.values())

Counter({'spanDestGrp': 3,
         'spanSrcGrp': 3,
         'spanSrc': 3,
         'mgmtMaintP': 3,
         'spanRsSrcToPathEp': 2,
         'fvSubnet': 2,
         'fvRsPathAtt': 2,
         'vzBrCP': 2,
         'dhcpRelayP': 2,
         'infraRsVlanNs': 2,
         'fvRsSecInherited': 2,
         'l1PhysIf': 2,
         'l2extInstP': 2,
         'mgmtOoB': 2,
         'leakInternalSubnet': 1,
         'leakRoutes': 1,
         'leakTo': 1,
         'aaaRbacClassPriv': 1,
         'aaaDomain': 1,
         'aaaRole': 1,
         'aaaSshAuth': 1,
         'aaaUser': 1,
         'aaaUserCert': 1,
         'aaaUserDomain': 1,
         'aaaUserRole': 1,
         'infraHPortS, infraPortBlk': 1,
         'infraHPortS, infraRsAccBaseGrp, infraPortBlk': 1,
         'spanFilterGrp': 1,
         'spanFilterEntry': 1,
         'infraHPortS, infraSubPortBlk': 1,
         'infraAttEntityP, infraProvAcc': 1,
         'infraRsDomP': 1,
         'infraRsFuncToEpg': 1,
         'fvAp': 1,
         

In [18]:
from defaultClassAttrValues import defaults

In [27]:
def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', "", "::", ":all:"]
    
    # here define exception list - handle dupes in values of class mappings
    # key (class) : value (dict) -> key (parent class) : value (correct mapping)

    exception_list = ['spanDestGrp',
                        'spanSrcGrp',
                        'spanSrc',
                        'mgmtMaintP',
                        'spanRsSrcToPathEp',
                        'fvSubnet',
                        'fvRsPathAtt',
                        'vzBrCP',
                        'dhcpRelayP',
                        'infraRsVlanNs',
                        'fvRsSecInherited',
                        'l1PhysIf',
                        'l2extInstP',
                        'mgmtOoB']

    exception_map = {
        # 'spanDestGrp': None,
        # 'spanSrcGrp': None,
        # 'spanSrc': None,
        # 'mgmtMaintP': None,
        # 'spanRsSrcToPathEp': None,
        'fvSubnet': {'aci_epg': 'aci_epg_subnet',
                     'aci_bd': 'aci_bd_subnet'},
        # 'fvRsPathAtt': None,
        'vzBrCP': "aci_contract",
        # 'dhcpRelayP': None,
        # 'infraRsVlanNs': None,
        # 'fvRsSecInherited': None,
        # 'l1PhysIf': None,
        # 'l2extInstP': None,
        # 'mgmtOoB': None
        }
    
    # helper method to navigate the exception dictionary
    def map_if_exception(child, parent = None):
        out = None
        try:
            out = exception_list[child][parent]
        except(KeyError):
            out = exception_list[child] # means there is no exception with the parent

        return out

    # simple method to check if a key in in the exception list
    # need to run this stuff before mapping anything - mapping is ambiguous
    def isexception(key):
        return key in exception_list

    def isdefault(parent_key, key, value, map):
        try:
            return value == map[parent_key][key] # means attribute has a default value
        except(KeyError):
            return False

    # process data recursively
    def remove_attributes_field(data, parent_key = None, attribute_parent_key = None, default_map = defaults):
        changes = []

        if isinstance(data, dict):
            keys_to_delete = []

            total_params = len(data.values())
            count_default = 0

            for key, value in data.items():

                if key == "attributes":
                    attribute_parent_key = parent_key
                    for attr_key, attr_value in value.items(): # here empty values are also handled
                        if isdefault(attribute_parent_key, attr_key, attr_value, default_map):
                            print(attribute_parent_key, attr_key, attr_value)
                            count_default += 1
                        if attr_value not in default_args and not isdefault(attribute_parent_key, attr_key, attr_value, defaults):
                            changes.append((attribute_parent_key, attr_key, attr_value))
                    keys_to_delete.append("attributes") # can append key too

                if isinstance(value, (dict, list)):
                    try:
                        remove_attributes_field(value, key, attribute_parent_key)
                    except (KeyError, TypeError) as e:
                        print(f"Error processing key {key}: {e}")

                if isinstance(value, dict) and not value:
                    keys_to_delete.append(key)

                remove_isNotConfigurable(key, keys_to_delete)

            for key in keys_to_delete:
                del data[key]

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    remove_attributes_field(item, parent_key, attribute_parent_key)

        # move all the "attributes" key-value pairs up
        for change in changes:
            parent_key, attr_key, attr_value = change # "change" is a 3x tuple
            if parent_key is not None and parent_key in data:
                data[parent_key][attr_key] = attr_value
            else:
                data[attr_key] = attr_value

    # process the initial data
    remove_attributes_field(data)

    # prune_dict(data)

    # Save the resulting dictionary to a YAML file
    with open(save_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    print(f"YAML file has been saved to {save_path}")

    return data

In [28]:
with open("tn-nils-4.json", 'r') as file:
    y = json.load(file)

In [29]:
reconstruct_yml(y)

fvRsTenantMonPol userdom all
fvEpTags userdom all
fvCtx bdEnforcedEnable no
fvCtx ipDataPlaneLearning enabled
fvCtx knwMcastAct permit
fvCtx pcEnfDir ingress
fvCtx pcEnfPref enforced
fvRsVrfValidationPol userdom all
fvRsOspfCtxPol userdom all
fvRsCtxToEpRet userdom all
fvRsCtxToExtRouteTagPol userdom all
fvRsBgpCtxPol userdom all
fvBD OptimizeWanBandwidth no
fvBD arpFlood no
fvBD epClear no
fvBD hostBasedRouting no
fvBD intersiteBumTrafficAllow no
fvBD intersiteL2Stretch no
fvBD ipLearning yes
fvBD ipv6McastAllow no
fvBD limitIpLearnToSubnets yes
fvBD mcastARPDrop yes
fvBD mcastAllow no
fvBD multiDstPktAct bd-flood
fvBD type regular
fvBD unicastRoute yes
fvBD unkMacUcastAct proxy
fvBD unkMcastAct flood
fvBD v6unkMcastAct flood
fvBD vmac not-applicable
fvSubnet ctrl nd
fvSubnet ipDPLearning enabled
fvSubnet preferred no
fvSubnet virtual no
fvRsMldsn userdom all
fvRsIgmpsn userdom all
fvRsCtx userdom all
fvRsBdToEpRet resolveAct resolve
fvRsBdToEpRet userdom all
fvRsBDToNdP userdom all
Y

{'fvTenant': {'children': [{'vnsSvcCont': {'userdom': 'all'}},
   {},
   {},
   {'fvCtx': {'children': [{},
      {'vzAny': {'matchT': 'AtleastOne',
        'prefGrMemb': 'disabled',
        'userdom': 'all'}},
      {},
      {},
      {},
      {}],
     'annotation': 'orchestrator:ansible',
     'descr': 'marcel',
     'name': 'bob',
     'vrfIndex': '0'}},
   {'fvBD': {'children': [{'fvSubnet': {'annotation': 'orchestrator:ansible',
        'ip': '10.10.10.1/24',
        'scope': 'public,shared'}},
      {},
      {},
      {'fvRsCtx': {'tnFvCtxName': 'bob'}},
      {},
      {}],
     'annotation': 'orchestrator:ansible',
     'enableRogueExceptMac': 'no',
     'mac': '00:22:BD:F8:19:FF',
     'name': 'tintin'}}],
  'annotation': 'orchestrator:ansible',
  'dn': 'uni/tn-nils',
  'name': 'nils'}}

In [23]:
import os
import json
import pprint

PATH_TO_JSON_DATA = "/Users/lpaggen/Documents/research_project/ACIBILIZER/DATAMODELS/fvClasses"

# this script allows to get a dictionary containing all default attribute values for classes
# its use is to determine which attributes to delete entirely when rebuilding yml playbooks

def parse_attr_defaults(dir):
    out = {} # output
    
    yml_bool_list = ['false', 'False', 'off', 'Off', 'No', 'true', 'True', 'on', 'On', 'Yes']

    yml_bool_map = {'false': 'no', # should cover all boolean cases (?)
                    'off': 'no',
                    'False': 'no',
                    'Off': 'no',
                    'No': 'no',
                    'true': 'yes',
                    'on': 'yes',
                    'True': 'yes',
                    'On': 'yes'}

    for file in os.listdir(dir): # read from the json file
        entry_dict = {} # serves as nested dict in "out"
        file_path = os.path.join(dir, file)
        with open(file_path, 'r') as file_:
            data = json.load(file_)
            # will simply need to do a switch to address other prefixes, it is the same logic
            className = "fv:" + file.split(".")[0] # gets the fv:xxxx format in the json
            for key, value in data[className]['properties'].items():
                try:
                    result = data[className]['properties'][key]["default"]
                    if data[className]['properties'][key]["default"] in yml_bool_list: # yml -> "true" == "yes", etc
                        result = yml_bool_map[data[className]['properties'][key]["default"]]
                    entry_dict[key] = result # sets value to default
                except(KeyError):
                    entry_dict[key] = None

        out[className.replace(":", "")] = entry_dict
        
    return out

g = parse_attr_defaults(PATH_TO_JSON_DATA)

# save to file
with open("defaultClassAttrValues.py", 'w') as file:
    file.write("defaults = ")
    pprint.pprint(g, stream=file)

print(f"Dictionary has been saved to defaultClassAttrValues.py")


Dictionary has been saved to defaultClassAttrValues.py


# regex testing 

In [104]:
def get_file_paths(dir):
    py_files = [] # save files
    for file in os.listdir(dir):
        file_path = str(dir) + r'/' + str(file)
        py_files.append(file_path)
    return py_files

def exceptionListParser(dir):

    py_files = get_file_paths(dir)
    
    out = {}

    for file_path in py_files:
        with open(file_path, 'r') as file:
            file_content = file.read()

        # regex patterns to find block of code within "child_config[]"
        pattern = r'child_configs\s*=\s*\[(.*?)\]'
        match = re.search(pattern, file_content, re.DOTALL)

        if match:
            extracted_content = match.group(1)

            # pattern to find all variables in quotes, and all variables not in quotes - per line
            quotes_pattern = re.compile(r'"([^"]+)"')
            variables_pattern = re.compile(r'\b\w+\b')

            entry_dict = {}  # nested dictionary 

            for line in extracted_content.split('\n'):
                stripped_line = line.strip()

                # skipping empty lines in match
                if not stripped_line:
                    continue

                quoted_strings = quotes_pattern.findall(line)

                all_words = variables_pattern.findall(line)
                non_quoted_vars = [word for word in all_words if word not in quoted_strings]

                # construct nested dictionaries, if more than 1 variable, just save both
                if len(non_quoted_vars) == 1 and len(quoted_strings) == 3:
                    entry_dict[non_quoted_vars[0]] = quoted_strings[0] + "[" + quoted_strings[-1] + "]"

                elif len(non_quoted_vars) == 2 and len(quoted_strings) == 4:
                    entry_dict[non_quoted_vars[0]] = quoted_strings[0] + "[" + quoted_strings[-2] + "]"
                    entry_dict[non_quoted_vars[1]] = quoted_strings[0] + "[" + quoted_strings[-1] + "]"

            out[file_path.split("/")[-1][:-3]] = entry_dict if entry_dict else None

        else: # find no match in the files (due to structure sometimes, double check)
            out[file_path.split("/")[-1][:-3]] = None

    return out

xyz = exceptionListParser("/Users/lpaggen/Documents/research_project/ansible-aci/modules")

# save to file
with open("exceptionDict.py", 'w') as file:
    file.write("excpetions = ")
    pprint.pprint(xyz, stream=file)

print(f"Dictionary has been saved to exceptionDict.py")

Dictionary has been saved to exceptionDict.py


In [102]:
exceptionListParser("/Users/lpaggen/Documents/research_project/ansible-aci/modules")

{'aci_fabric_pod_policy_group': None,
 'aci_esg_contract_master': None,
 'aci_l3out_floating_svi': None,
 'aci_l3out_bgp_peer': None,
 'aci_bgp_timers_policy': None,
 'aci_l2out': None,
 'aci_l3out_hsrp_group': None,
 'aci_subject_label': None,
 'aci_interface_policy_bfd': None,
 'aci_vlan_pool': None,
 'aci_cloud_external_epg': None,
 'aci_domain_to_encap_pool': None,
 'aci_tenant_span_src_group_to_dst_group': None,
 'aci_cloud_epg': None,
 'aci_vmm_vswitch_policy': None,
 'aci_access_spine_interface_profile_to_spine_switch_profile': None,
 'aci_fabric_pod': None,
 'aci_fabric_switch_policy_group': None,
 'aci_l3out_logical_interface_profile_ospf_policy': None,
 'aci_system_banner': None,
 'aci_contract_export': None,
 'aci_domain_to_vlan_pool': {'aci_mo': 'infraRsVlanNs[tDn]'},
 'aci_l3out_hsrp_interface_profile': None,
 'aci_l3out_extsubnet': None,
 'aci_key_policy': None,
 'aci_l3out_logical_interface_profile': None,
 'aci_l3out_static_routes': None,
 'aci_interface_policy_pim': No